### Task 1


---



In [24]:
def generate_maze(rows, cols):
    # Initialize a maze with all cells set to '.'
    maze = [['.' for _ in range(cols)] for _ in range(rows)]

    # Randomly select the starting node in the first 2 columns
    start_col = random.randint(0, 1)
    start_row = random.randint(0, rows - 1)
    maze[start_row][start_col] = 'S'

    # Randomly select the goal node in the last 2 columns
    goal_col = random.randint(cols - 2, cols - 1)
    goal_row = random.randint(0, rows - 1)
    maze[goal_row][goal_col] = 'G'

    # Randomly select four barrier nodes from the remaining nodes
    barrier_nodes = []
    barrier_count = 0
    while barrier_count < 4:
        col = random.randint(0, cols - 1)
        row = random.randint(0, rows - 1)
        if maze[row][col] == '.' and (col != start_col or row != start_row) and (col != goal_col or row != goal_row) and (col, row) not in barrier_nodes:
            maze[row][col] = '#'
            barrier_nodes.append((col, row))
            barrier_count += 1

    return maze, (start_col, start_row), (goal_col, goal_row), barrier_nodes

### Task 2


---



In [56]:
def dfs_maze(maze, start_node, goal_node, barrier_nodes):
    visited = []
    stack = []
    parent = {}
    stack.append(start_node)
    visited.append(start_node)

    while stack and goal_node not in visited:
        node = stack.pop()
        neighbors = get_neighbors(maze, node, barrier_nodes)
        for neighbor in neighbors:
            if neighbor not in visited:
                stack.append(neighbor)
                visited.append(neighbor)
                parent[neighbor] = node

    if goal_node in visited:
        path = []
        current = goal_node
        while current != start_node:
            path.insert(0, current)
            current = parent[current]
        path.insert(0, start_node)
        return visited, len(visited), path
    else:
        return visited, len(visited), None

def get_neighbors(maze, node, barrier_nodes):
    row, col = node
    rows, cols = len(maze), len(maze[0])
    neighbors = []

    # Define the order for processing neighbors
    processing_order = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

    for i, j in processing_order:
        new_row, new_col = row + i, col + j
        if 0 <= new_row < rows and 0 <= new_col < cols and (new_col, new_row) not in barrier_nodes:
            neighbors.append((new_col, new_row))

    return neighbors


### Task 3

In [71]:
# Manhattan distance calculation function
def calculate_manhattan_distance(node, goal):
    Nx, Ny = node
    Gx, Gy = goal
    return abs(Nx - Gx) + abs(Ny - Gy)

# Function to calculate heuristic cost for all nodes in the maze
def calculate_heuristic_for_all_nodes(maze, goal):
    rows, cols = len(maze), len(maze[0])
    heuristic_costs = []

    for y in range(rows):
        for x in range(cols):
            if maze[y][x] != '#':
                node = (x, y)
                heuristic_cost = calculate_manhattan_distance(node, goal)
                heuristic_costs.append(((x, y), heuristic_cost))

    return heuristic_costs

### Task 4


---



In [78]:
def a_star_maze(maze, start_node, goal_node, barrier_nodes):
    visited = []
    queue = []
    parent = {}
    g_score = {start_node: 0}
    f_score = {start_node: calculate_manhattan_distance(start_node, goal_node)}
    queue.append(start_node)

    while queue:
        current = queue.pop(0)
        if current == goal_node:
            path = []
            while current != start_node:
                path.insert(0, current)
                current = parent[current]
            path.insert(0, start_node)
            return visited, len(visited), path

        visited.append(current)
        neighbors = get_neighbors(maze, current, barrier_nodes)
        for neighbor in neighbors:
            tentative_g_score = g_score[current] + 1
            if neighbor in visited and tentative_g_score >= g_score.get(neighbor, float('inf')):
                continue
            if tentative_g_score < g_score.get(neighbor, float('inf')):
                parent[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + calculate_manhattan_distance(neighbor, goal_node)
                queue.append(neighbor)
                queue.sort(key=lambda x: f_score[x])

    return visited, None, None

### Final Outputs


---



In [79]:
def main():
  # Define maze dimensions
  rows, cols = 6, 6

  # Call the function to generate the maze and get the results
  maze, start_node, goal_node, barrier_nodes = generate_maze(rows, cols)

  # Call the dfs_maze function and print the results
  visited, time, path = dfs_maze(maze, start_node, goal_node, barrier_nodes)

  # Display the generated maze
  print("\nMaze:")
  for row in maze:
      print(' '.join(row))

  # Display additional information
  print("\nStart Node:", start_node)
  print("Goal Node:", goal_node)
  print("Barrier Nodes:", barrier_nodes)

  print("\nDFS")
  print("\nVisited Nodes:", visited)
  print("Time to Find the Goal:", time, "minutes")
  print("Final Path:", path)

  print("\nVizualization of the Visited Nodes in DFS")
  visited_n = set()
  for steps in visited:
    visited_n.add(steps)
  for i in range(len(maze)):
    row = ['1' if (i, j) in visited_n else '0' for j in range(len(maze[0]))]
    print(' '.join(row))

  heuristic_costs = calculate_heuristic_for_all_nodes(maze, goal_node)

  print("\nHeuristic Costs for All Nodes:")
  for node, cost in heuristic_costs:
      print(f"Node {node}: {cost}")

  # Call the a_star_maze function and print the results
  visitedA, timeA, pathA = a_star_maze(maze, start_node, goal_node, barrier_nodes)

  print("\nA*")

  print("\nVisited Nodes:", visitedA)
  if timeA is not None:
      print("Time to Find the Goal:", timeA, "minutes")
      print("Final Path:", pathA)
  else:
      print("Goal could not be reached.")

  print("\nVizualization of the Visited Nodes in A*")
  visited_nd = set()
  for steps in visitedA:
    visited_nd.add(steps)
  for i in range(len(maze)):
    row = ['1' if (i, j) in visited_nd else '0' for j in range(len(maze[0]))]
    print(' '.join(row))

if __name__ == "__main__":
    main()


Maze:
. . . . G .
. . . . # .
. . # . . .
. . # . . .
S . . . # .
. . . . . .

Start Node: (0, 4)
Goal Node: (4, 0)
Barrier Nodes: [(4, 4), (2, 2), (4, 1), (2, 3)]

DFS

Visited Nodes: [(0, 4), (5, 0), (3, 0), (5, 1), (3, 1), (0, 3), (1, 4), (1, 2), (2, 4), (0, 2), (1, 0), (2, 1), (1, 1), (0, 1), (2, 0), (0, 0), (1, 3), (3, 2), (4, 2), (4, 0)]
Time to Find the Goal: 20 minutes
Final Path: [(0, 4), (3, 1), (0, 2), (1, 1), (2, 0), (1, 3), (4, 0)]

Vizualization of the Visited Nodes in DFS
1 1 1 1 1 0
1 1 1 1 1 0
1 1 0 0 1 0
1 1 1 0 0 0
1 0 1 0 0 0
1 1 0 0 0 0

Heuristic Costs for All Nodes:
Node (0, 0): 4
Node (1, 0): 3
Node (2, 0): 2
Node (3, 0): 1
Node (4, 0): 0
Node (5, 0): 1
Node (0, 1): 5
Node (1, 1): 4
Node (2, 1): 3
Node (3, 1): 2
Node (5, 1): 2
Node (0, 2): 6
Node (1, 2): 5
Node (3, 2): 3
Node (4, 2): 2
Node (5, 2): 3
Node (0, 3): 7
Node (1, 3): 6
Node (3, 3): 4
Node (4, 3): 3
Node (5, 3): 4
Node (0, 4): 8
Node (1, 4): 7
Node (2, 4): 6
Node (3, 4): 5
Node (5, 4): 5
Node (0, 5): 

In [80]:
if __name__ == "__main__":
    main()


Maze:
. . . . . .
. . . . . .
. . . . . .
. . . . . #
. . # # . G
. S # . . .

Start Node: (1, 5)
Goal Node: (5, 4)
Barrier Nodes: [(2, 5), (5, 3), (2, 4), (3, 4)]

DFS

Visited Nodes: [(1, 5), (4, 1), (5, 2), (5, 0), (4, 2), (4, 0), (1, 4), (0, 5), (0, 3), (1, 3), (2, 1), (3, 2), (3, 0), (2, 2), (2, 0), (1, 2), (0, 1), (1, 1), (1, 0), (0, 2), (0, 0), (3, 1), (2, 3), (0, 4), (5, 1), (3, 3), (4, 3), (4, 4), (3, 5), (4, 5), (5, 5), (5, 4)]
Time to Find the Goal: 32 minutes
Final Path: [(1, 5), (4, 0), (1, 3), (2, 0), (1, 1), (0, 2), (3, 1), (2, 3), (4, 3), (4, 5), (5, 5), (5, 4)]

Vizualization of the Visited Nodes in DFS
1 1 1 1 1 1
1 1 1 1 1 1
1 1 1 1 0 0
1 1 1 1 0 1
1 1 1 1 1 1
1 1 1 0 1 1

Heuristic Costs for All Nodes:
Node (0, 0): 9
Node (1, 0): 8
Node (2, 0): 7
Node (3, 0): 6
Node (4, 0): 5
Node (5, 0): 4
Node (0, 1): 8
Node (1, 1): 7
Node (2, 1): 6
Node (3, 1): 5
Node (4, 1): 4
Node (5, 1): 3
Node (0, 2): 7
Node (1, 2): 6
Node (2, 2): 5
Node (3, 2): 4
Node (4, 2): 3
Node (5, 2):

In [81]:
if __name__ == "__main__":
    main()


Maze:
. . . . . #
. . . . . .
. . . . # .
S . . . . G
. . . . . .
. . # . # .

Start Node: (0, 3)
Goal Node: (5, 3)
Barrier Nodes: [(2, 5), (4, 5), (5, 0), (4, 2)]

DFS

Visited Nodes: [(0, 3), (4, 0), (2, 0), (3, 1), (4, 1), (2, 1), (2, 2), (0, 2), (1, 3), (1, 1), (2, 3), (0, 1), (0, 0), (1, 0), (1, 2), (3, 2), (3, 0), (0, 4), (1, 4), (5, 1), (5, 2), (3, 5), (1, 5), (2, 4), (3, 4), (5, 3), (3, 3), (4, 4), (5, 4)]
Time to Find the Goal: 29 minutes
Final Path: [(0, 3), (2, 1), (0, 1), (0, 0), (1, 0), (1, 2), (3, 0), (1, 4), (5, 2), (3, 4), (5, 3)]

Vizualization of the Visited Nodes in DFS
1 1 1 1 1 0
1 1 1 1 1 1
1 1 1 1 1 0
1 1 1 1 1 1
1 1 0 0 1 0
0 1 1 1 1 0

Heuristic Costs for All Nodes:
Node (0, 0): 8
Node (1, 0): 7
Node (2, 0): 6
Node (3, 0): 5
Node (4, 0): 4
Node (0, 1): 7
Node (1, 1): 6
Node (2, 1): 5
Node (3, 1): 4
Node (4, 1): 3
Node (5, 1): 2
Node (0, 2): 6
Node (1, 2): 5
Node (2, 2): 4
Node (3, 2): 3
Node (5, 2): 1
Node (0, 3): 5
Node (1, 3): 4
Node (2, 3): 3
Node (3, 3): 2